- need to idetify which features we want to keep because we have too much raw data atm
- a lot of the files are also gzipped so i need to unzip them to see what goodies are inside


# BRON data (accidents)

In [1]:
import json
import pandas as pd
from pathlib import Path

In [12]:
root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / ".git").exists()), Path.cwd())
base = root / "data_raw" / "BRON data"

def load_year_df(year: int) -> pd.DataFrame:
    fp = base / f"ongevallen_{year}.json"
    with fp.open("r", encoding="utf-8") as f:
        data = json.load(f)
    rows = []
    for feat in data.get("features", []):
        props = feat.get("properties", {}).copy()
        rows.append(props)
    df = pd.DataFrame(rows)
    print(f"{year} shape: {df.shape}")
    return df

df_2022 = load_year_df(2022)
df_2023 = load_year_df(2023)
df_2024 = load_year_df(2024)

print("\n2022 columns:", df_2022.columns.tolist())
print("2023 columns:", df_2023.columns.tolist())
print("2024 columns:", df_2024.columns.tolist())

2022 shape: (122036, 38)
2023 shape: (134005, 38)
2024 shape: (126380, 38)

2022 columns: ['verkeersongeval_nummer', 'jaar_ongeval', 'verkeersongeval_afloop', 'aantal_partijen', 'partij_1_objecttype', 'partij_1_objecttype_overig', 'partij_2_objecttype', 'partij_2_objecttype_overig', 'aard_ongeval', 'niveau_koppelen', 'wegsituatie', 'bebouwde_kom', 'maximum_snelheid', 'wegverlichting', 'wegverharding', 'wegdek', 'lichtgesteldheid', 'zichtafstand', 'weersgesteldheid', 'bijz_verkeersmaatregel', 'bijz_verkeersmaatregel_overig', 'bijz_infrastructuur', 'bijz_infrastructuur_overig', 'bijz_tijdelijke_aard', 'bijz_tijdelijke_aard_overig', 'junctie_id', 'wegvak_id', 'hectometer', 'straatnaam', 'woonplaats', 'actueel', 'wegbeheerder', 'gemeente', 'provincie', 'dienstnaam', 'districtnaam', 'indicatie_alcohol', 'gdb_geomattr_data']
2023 columns: ['verkeersongeval_nummer', 'jaar_ongeval', 'verkeersongeval_afloop', 'aantal_partijen', 'partij_1_objecttype', 'partij_1_objecttype_overig', 'partij_2_obje

strange?

In [13]:
cols_2022 = set(df_2022.columns)
cols_2023 = set(df_2023.columns)
cols_2024 = set(df_2024.columns)

print("Columns in all years:")
print(cols_2022 & cols_2023 & cols_2024)

print("\nColumns only in 2022:")
print(cols_2022 - cols_2023 - cols_2024)

print("\nColumns only in 2023:")
print(cols_2023 - cols_2022 - cols_2024)

print("\nColumns only in 2024:")
print(cols_2024 - cols_2022 - cols_2023)

Columns in all years:
{'gemeente', 'wegsituatie', 'bijz_tijdelijke_aard_overig', 'bijz_tijdelijke_aard', 'partij_1_objecttype_overig', 'indicatie_alcohol', 'gdb_geomattr_data', 'bijz_infrastructuur', 'junctie_id', 'partij_1_objecttype', 'verkeersongeval_afloop', 'aantal_partijen', 'provincie', 'partij_2_objecttype', 'dienstnaam', 'hectometer', 'wegverharding', 'lichtgesteldheid', 'straatnaam', 'wegvak_id', 'aard_ongeval', 'woonplaats', 'wegverlichting', 'actueel', 'weersgesteldheid', 'wegbeheerder', 'niveau_koppelen', 'bebouwde_kom', 'wegdek', 'bijz_infrastructuur_overig', 'partij_2_objecttype_overig', 'verkeersongeval_nummer', 'zichtafstand', 'maximum_snelheid', 'jaar_ongeval', 'bijz_verkeersmaatregel_overig', 'bijz_verkeersmaatregel', 'districtnaam'}

Columns only in 2022:
set()

Columns only in 2023:
set()

Columns only in 2024:
set()


greaaaat 

# check missing values

In [14]:
# Missing values count for 2022
missing_2022 = df_2022.isnull().sum()
missing_pct_2022 = (missing_2022 / len(df_2022)) * 100

missing_df_2022 = pd.DataFrame({
    'Column': missing_2022.index,
    'Missing_Count': missing_2022.values,
    'Missing_Percentage': missing_pct_2022.values
})

# Sort by missing percentage
missing_df_2022 = missing_df_2022.sort_values('Missing_Percentage', ascending=False)
missing_df_2022

,Column,Missing_Count,Missing_Percentage
37,gdb_geomattr_data,122036,100.000000
17,zichtafstand,122036,100.000000
20,bijz_verkeersmaatregel_overig,122036,100.000000
23,bijz_tijdelijke_aard,121576,99.623062
24,bijz_tijdelijke_aard_overig,121187,99.304304
5,partij_1_objecttype_overig,120687,98.894588
36,indicatie_alcohol,119822,98.185781
7,partij_2_objecttype_overig,119298,97.756400
21,bijz_infrastructuur,117092,95.948736
19,bijz_verkeersmaatregel,114647,93.945229


In [15]:
# Missing values count for 2023
missing_2023 = df_2023.isnull().sum()
missing_pct_2023 = (missing_2023 / len(df_2023)) * 100

missing_df_2023 = pd.DataFrame({
    'Column': missing_2023.index,
    'Missing_Count': missing_2023.values,
    'Missing_Percentage': missing_pct_2023.values
})

missing_df_2023 = missing_df_2023.sort_values('Missing_Percentage', ascending=False)
missing_df_2023

,Column,Missing_Count,Missing_Percentage
37,gdb_geomattr_data,134005,100.000000
17,zichtafstand,134005,100.000000
20,bijz_verkeersmaatregel_overig,134001,99.997015
24,bijz_tijdelijke_aard_overig,133239,99.428380
23,bijz_tijdelijke_aard,132723,99.043319
36,indicatie_alcohol,130924,97.700832
21,bijz_infrastructuur,130436,97.336667
7,partij_2_objecttype_overig,130232,97.184433
19,bijz_verkeersmaatregel,126386,94.314391
22,bijz_infrastructuur_overig,125697,93.800231


In [16]:
# Missing values count for 2024
missing_2024 = df_2024.isnull().sum()
missing_pct_2024 = (missing_2024 / len(df_2024)) * 100

missing_df_2024 = pd.DataFrame({
    'Column': missing_2024.index,
    'Missing_Count': missing_2024.values,
    'Missing_Percentage': missing_pct_2024.values
})

missing_df_2024 = missing_df_2024.sort_values('Missing_Percentage', ascending=False)
missing_df_2024

,Column,Missing_Count,Missing_Percentage
37,gdb_geomattr_data,126380,100.000000
17,zichtafstand,126380,100.000000
20,bijz_verkeersmaatregel_overig,126372,99.993670
24,bijz_tijdelijke_aard_overig,125582,99.368571
23,bijz_tijdelijke_aard,125056,98.952366
36,indicatie_alcohol,122925,97.266181
21,bijz_infrastructuur,122836,97.195759
7,partij_2_objecttype_overig,122366,96.823865
19,bijz_verkeersmaatregel,118678,93.905681
22,bijz_infrastructuur_overig,118446,93.722108


In [17]:
# Combine all years
df_all = pd.concat([df_2022, df_2023, df_2024], ignore_index=True)

print(f"Total accidents (2022-2024): {len(df_all):,}")
print(f"Total columns: {len(df_all.columns)}")
print(f"\nShape: {df_all.shape}")

Total accidents (2022-2024): 382,421
Total columns: 38

Shape: (382421, 38)


In [18]:
# Missing values for combined data
missing_all = df_all.isnull().sum()
missing_pct_all = (missing_all / len(df_all)) * 100

missing_df_all = pd.DataFrame({
    'Column': missing_all.index,
    'Missing_Count': missing_all.values,
    'Missing_Percentage': missing_pct_all.values
})

missing_df_all = missing_df_all.sort_values('Missing_Percentage', ascending=False)
missing_df_all

,Column,Missing_Count,Missing_Percentage
37,gdb_geomattr_data,382421,100.000000
17,zichtafstand,382421,100.000000
20,bijz_verkeersmaatregel_overig,382409,99.996862
24,bijz_tijdelijke_aard_overig,380008,99.369020
23,bijz_tijdelijke_aard,379355,99.198266
36,indicatie_alcohol,373671,97.711946
7,partij_2_objecttype_overig,371896,97.247798
21,bijz_infrastructuur,370364,96.847192
19,bijz_verkeersmaatregel,359711,94.061519
22,bijz_infrastructuur_overig,356847,93.312606


| Dutch                             | English                          |
| --------------------------------- | -------------------------------- |
| **gdb_geomattr_data**             | geometric attribute data         |
| **zichtafstand**                  | visibility distance              |
| **bijz_verkeersmaatregel_overig** | special traffic measure (other)  |
| **bijz_tijdelijke_aard_overig**   | special temporary nature (other) |
| **bijz_tijdelijke_aard**          | special temporary nature         |
| **indicatie_alcohol**             | indication of alcohol            |
| **partij_2_objecttype_overig**    | party 2 object type (other)      |
| **bijz_infrastructuur**           | special infrastructure           |
| **bijz_verkeersmaatregel**        | special traffic measure          |
| **bijz_infrastructuur_overig**    | special infrastructure (other)   |


# data types

In [19]:
df_all.dtypes

verkeersongeval_nummer             int64
jaar_ongeval                       int64
verkeersongeval_afloop            object
aantal_partijen                    int64
partij_1_objecttype               object
partij_1_objecttype_overig        object
partij_2_objecttype               object
partij_2_objecttype_overig        object
aard_ongeval                      object
niveau_koppelen                   object
wegsituatie                       object
bebouwde_kom                      object
maximum_snelheid                 float64
wegverlichting                    object
wegverharding                     object
wegdek                            object
lichtgesteldheid                  object
zichtafstand                      object
weersgesteldheid                  object
bijz_verkeersmaatregel            object
bijz_verkeersmaatregel_overig     object
bijz_infrastructuur               object
bijz_infrastructuur_overig        object
bijz_tijdelijke_aard              object
bijz_tijdelijke_

In [20]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 382421 entries, 0 to 382420
Data columns (total 38 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   verkeersongeval_nummer         382421 non-null  int64  
 1   jaar_ongeval                   382421 non-null  int64  
 2   verkeersongeval_afloop         382421 non-null  object 
 3   aantal_partijen                382421 non-null  int64  
 4   partij_1_objecttype            326056 non-null  object 
 5   partij_1_objecttype_overig     56365 non-null   object 
 6   partij_2_objecttype            256122 non-null  object 
 7   partij_2_objecttype_overig     10525 non-null   object 
 8   aard_ongeval                   382421 non-null  object 
 9   niveau_koppelen                382421 non-null  object 
 10  wegsituatie                    251703 non-null  object 
 11  bebouwde_kom                   254103 non-null  object 
 12  maximum_snelheid              

In [21]:
df_all.head(10)

,verkeersongeval_nummer,jaar_ongeval,verkeersongeval_afloop,aantal_partijen,partij_1_objecttype,partij_1_objecttype_overig,partij_2_objecttype,partij_2_objecttype_overig,aard_ongeval,niveau_koppelen,...,straatnaam,woonplaats,actueel,wegbeheerder,gemeente,provincie,dienstnaam,districtnaam,indicatie_alcohol,gdb_geomattr_data
0,20220001580,2022,Uitsluitend materiele schade,2,Trekker,None,None,AANHANGWAGEN,Onbekend,Ongeval gekoppeld op straat niveau.,...,Om den Noort,Wolvega,Ja,Gemeente,Weststellingwerf,Friesland,None,None,None,None
1,20220001286,2022,Uitsluitend materiele schade,2,Personenauto,None,Personenauto,None,Kop/staart,Ongeval exact gekoppeld op wegvak of junctie a...,...,Blijhamsterweg,Winschoten,Ja,Gemeente,Oldambt,Groningen,None,None,None,None
2,20220000619,2022,Uitsluitend materiele schade,3,Personenauto,None,Overig wegmeubilair,None,Vast voorwerp,Ongeval exact gekoppeld op wegvak of junctie a...,...,Engelkeslaan,Westerlee,Ja,Gemeente,Oldambt,Groningen,None,None,None,None
3,20220004648,2022,Uitsluitend materiele schade,2,Personenauto,None,Personenauto,None,Onbekend,Ongeval gekoppeld op straat niveau.,...,Socratesstraat,Apeldoorn,Ja,Gemeente,Apeldoorn,Gelderland,None,None,None,None
4,20220001991,2022,Uitsluitend materiele schade,2,Personenauto,None,Overig wegmeubilair,None,Vast voorwerp,Ongeval exact gekoppeld op wegvak of junctie a...,...,RYKSWG,Veendam,Ja,Rijk,Veendam,Groningen,RWS Noord-Nederland,NN District Oost,None,None
5,20220002382,2022,Uitsluitend materiele schade,2,Personenauto,None,Personenauto,None,Onbekend,Ongeval exact gekoppeld op wegvak of junctie a...,...,Maarweg,Uithuizen,Ja,Gemeente,Het Hogeland,Groningen,None,None,None,None
6,20220000729,2022,Uitsluitend materiele schade,1,Trekker,None,None,None,Onbekend,Ongeval exact gekoppeld op wegvak of junctie a...,...,Kloosterdwarsweg,Warffum,Ja,Gemeente,Het Hogeland,Groningen,None,None,None,None
7,20220002409,2022,Uitsluitend materiele schade,2,Personenauto,None,Overig wegmeubilair,None,Vast voorwerp,Ongeval gekoppeld op straat niveau.,...,Auwemalaan,Leek,Ja,Gemeente,Westerkwartier,Groningen,None,None,None,None
8,20220002468,2022,Letsel,2,Bromfiets,None,Personenauto,None,Frontaal,Ongeval gekoppeld op straat niveau.,...,Petrus Campersingel,Groningen,Ja,Gemeente,Groningen,Groningen,None,None,None,None
9,20220002451,2022,Uitsluitend materiele schade,2,Personenauto,None,Personenauto,None,Onbekend,Ongeval exact gekoppeld op wegvak of junctie a...,...,Rijksweg,Beilen,Ja,Rijk,Midden-Drenthe,Drenthe,RWS Noord-Nederland,NN District Oost,None,None


In [22]:
df_all.describe()

,verkeersongeval_nummer,jaar_ongeval,aantal_partijen,maximum_snelheid,junctie_id,wegvak_id,hectometer
count,3.824210e+05,382421.000000,382421.000000,354836.000000,1.059490e+05,1.519720e+05,124500.000000
mean,2.023088e+10,2023.011359,1.888231,64.666917,3.769235e+08,3.963137e+08,598.893446
std,7.531171e+06,0.805891,0.796788,26.401739,1.657023e+08,1.724683e+08,574.849458
min,2.022000e+10,2022.000000,1.000000,15.000000,3.014103e+07,2.714203e+07,0.000000
25%,2.022907e+10,2022.000000,1.000000,50.000000,2.274301e+08,2.343930e+08,176.000000
50%,2.023008e+10,2023.000000,2.000000,50.000000,3.511950e+08,3.775085e+08,426.000000
75%,2.024004e+10,2024.000000,2.000000,100.000000,6.001437e+08,6.004702e+08,834.000000
max,2.024018e+10,2024.000000,19.000000,130.000000,6.005018e+08,6.013084e+08,3119.000000


# Cleaning BRON data

In [33]:
empty_columns = ['gdb_geomattr_data', 'zichtafstand']


files = [
    'BRON data/ongevallen_2022.json',
    'BRON data/ongevallen_2023.json',
    'BRON data/ongevallen_2024.json'
]


output_dir = Path(r'C:\Users\nicol\Documents\TrafficOntology_Project\TrafficOntology\data_processed\BRON_cleaned')
output_dir.mkdir(parents=True, exist_ok=True)


for filepath in files:
    filename = Path(filepath).name
    print(f"Processing {filename}...")
    

    with open(filepath, 'r', encoding='utf-8') as f:
        data = json.load(f)
    

    for feature in data['features']:
        for col in empty_columns:
            feature['properties'].pop(col, None)
    

    output_path = output_dir / filename.replace('.json', '_clean.json')
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, separators=(',', ':'))
    
    size_mb = output_path.stat().st_size / (1024 * 1024)

    #had to switch from indents to separators to lower file size 
    print(f"Saved to {output_path.name} ({size_mb:.1f} MB)\n")

print("Done yippie")

Processing ongevallen_2022.json...
Saved to ongevallen_2022_clean.json (137.2 MB)

Processing ongevallen_2023.json...
Saved to ongevallen_2023_clean.json (150.5 MB)

Processing ongevallen_2024.json...
Saved to ongevallen_2024_clean.json (141.9 MB)

Done yippie
